# PSO algorithm

In [1]:
# Imports

import numpy as np
import random



#Loading the datasets

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2020-03-02 08:26:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data.1’

iris.data.1         100%[===================>]   4.44K  --.-KB/s    in 0s      

2020-03-02 08:26:45 (81.8 MB/s) - ‘iris.data.1’ saved [4551/4551]



In [2]:
"""Iris dataset"""

iris_data = []

with open("iris.data") as iris:
    for line in iris.readlines():
        values = line.split(sep=",")
        if values[-1] == 'Iris-setosa\n':
            values[-1] = 0
        if values[-1] == 'Iris-versicolor\n':
            values[-1] = 1
        if values[-1] == 'Iris-virginica\n':
            values[-1] = 2
        iris_data.append(values)

iris_data.pop() #Remove appendixical 'empty' last entry

for j in range(len(iris_data)):
    for i in range(len(iris_data[j])):
        # values[i] = values[i].replace(".", ",")
        iris_data[j][i] = float(iris_data[j][i])

iris_x = [values[:len(values)-1] for values in iris_data]
iris_y = [values[-1] for values in iris_data]

iris_x = np.array(iris_x)
iris_y = np.array(iris_y)

print("Iris data set loaded, length is",len(iris_data))

del iris_data



"""Artificial dataset 1"""

x_ai1 = np.random.uniform(low = -1, high = 1, size = (400,2))
y_ai1 = []
for i in x_ai1:
    if (i[0] >= 0.7 or (i[0] <= 0.3 and (i[1] >= -0.2-i[0]))):
        y_ai1.append(1)
    else:
        y_ai1.append(0)

y_ai1 = np.array(y_ai1)

print("Artificial data set 1 loaded, length is", len(x_ai1))

Iris data set loaded, length is 150
Artificial data set 1 loaded, length is 400


In [0]:
class Particle():

    def __init__(self, centroids, Nc, Nd):
        self.v = np.random.rand(Nc, Nd)
        self.centroids = centroids
        self.local_best = np.copy(self.centroids)

    def update(self):
        v_next = omega * np.copy(self.v)
        global global_best
        # print("update 1:", v_next)
        v_next += a1 * np.random.rand(*np.shape(self.v)) * (self.local_best - self.centroids)
        # print("update 2:", v_next)
        v_next += a2 * np.random.rand(*np.shape(self.v)) * (global_best - self.centroids)
        # print("update 3:", v_next)
        self.v = v_next
        self.centroids += self.v

    # def copy(self):
    #     return Particle(np.copy(self.centroids))

        

def get_closest_cluster_index(point, centroids):
    distances = [np.linalg.norm(point-centroid) for centroid in centroids]
    # print("gcci centroids:", len(centroids))
    # print("gcci returns:",np.argmin(distances))
    return np.argmin(distances)

def PSO_Clustering(X, Y, swarm_size, epochs, Nc):

    """Initializations"""

    Nd = len(X[0])
    No = len(X)

    #Calculate the minimum and maximum values for each dimension of the data. This is used in the random initialization of centroids
    minis = [min([x[i] for x in X]) for i in range(Nd)]
    maxis = [max([x[i] for x in X]) for i in range(Nd)]

    #Initialize particles with centroids based on minimum and maximum values for each dimension
    swarm = np.asarray([Particle([[np.random.random() * (maxis[i] - minis[i]) + minis[i] for i in range(Nd)] for j in range(Nc)], Nc, Nd) for k in range(swarm_size)])

    global global_best
    global_best = np.array([[np.random.random() * (maxis[i] - minis[i]) + minis[i] for i in range(Nd)] for j in range(Nc)])

    def get_fitness(centroids):
        
        clusters = [[] for i in range(Nc)]

        #Assign points to clusters
        for x in X:
            clusters[get_closest_cluster_index(x, centroids)].append(x)

        #The fitness formula, or formula (8) in the paper
        #For each cluster, take the sum of the distances between the cluster's centroid and its datapoints and divide it by the number of datapoints in the cluster
        #Sum the cluster-sums and divide by the number of clusters

        return sum([sum([np.linalg.norm(x-centroids[i]) for x in clusters[i]])/(len(clusters[i])+1) for i in range(Nc)]) / Nc

    


    ############# Training Loop ###############

    for epoch in range(epochs):

        print("Starting epoch", epoch)

        for particle in swarm:

            #Calculate fitness and update fitness values
            fitness = get_fitness(particle.centroids)
            if fitness < get_fitness(global_best):
                global_best = np.copy(particle.centroids)
            if fitness < get_fitness(particle.local_best):
                particle.local_best = np.copy(particle.centroids)

            particle.update()

    return global_best, get_fitness(global_best)

In [5]:
omega=0.5
a1=0.5
a2=0.5

#del global_best
pso_iris_quants = []
for i in range(10):
    print(i)
    iris_best, iris_quant = PSO_Clustering(iris_x, iris_y, swarm_size=100, epochs=20, Nc=3)
    pso_iris_quants.append(iris_quant)
    print("Quant:", pso_iris_quants)
# print("Endbest:", iris_best)
print("Quant:", pso_iris_quants)

0
Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Quant: [0.6806216504824659]
1
Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Quant: [0.6806216504824659, 0.509899061549874]
2
Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoc

In [0]:
# Evaluation:

correct = 0
print("test global", iris_best)
for x, y in zip(iris_x, iris_y):

    prediction = get_closest_cluster_index(x, iris_best)
    print(prediction, y)
    if prediction == y:
        correct += 1

print("Eval:", correct, "out of", len(iris_x))

test global [[4.90262998 3.23645444 1.37823009 0.52658243]
 [6.1942492  2.81378164 5.11739008 1.59730599]
 [5.34738122 3.20491336 6.53705387 1.23740958]]
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
0 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
0 1.0
1 1.0
1 1.0
1 1.0
1 1.0
0 1.0
1 1.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0
1 2.0


In [13]:
pso_ai1_quants = []
for i in range(10):
    print(i)
    ai1_best, ai_quant = PSO_Clustering(x_ai1, y_ai1, swarm_size=100, epochs=20, Nc=2)
    pso_ai1_quants.append(ai_quant)
    print("ai1 quant: ", pso_ai1_quants)
# print("Endbest:", x_ai1)
print("ai1 quant: ", pso_ai1_quants)

0
Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
ai1 quant:  [0.5373681344871976]
1
Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
ai1 quant:  [0.5373681344871976, 0.539782686901401]
2
Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Sta

In [0]:

correct = 0
print("test global", ai1_best)
for x, y in zip(x_ai1, y_ai1):

    prediction = get_closest_cluster_index(x, ai1_best)
    print(prediction, y)
    if prediction == y:
        correct += 1

print("Eval:", correct, "out of", len(x_ai1))


test global [[0.23585948 0.15906304]
 [0.84316122 0.90574376]]
0 0
1 1
0 0
0 0
1 1
0 0
0 1
0 1
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 1
1 1
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 1
0 0
0 0
1 1
0 0
0 0
1 1
1 1
0 0
0 1
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 1
1 1
0 0
0 0
0 1
0 0
0 0
0 0
1 1
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
1 1
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 1
1 0
0 1
0 1
1 1
0 0
0 0
1 1
0 0
0 0
1 1
0 0
0 1
0 1
0 0
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 1
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 1
0 0
1 1
0 0
0 0
1 1
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
1 1
0 1
0 0
0 0
0 0
1 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 1
1 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 1
1

In [0]:
print("test")
print("mean iris", np.mean(pso_iris_quants))
print("std iris", np.std(pso_iris_quants))
print("mean iris", np.mean(pso_ai_quants))
print("std iris", np.std(pso_ai_quants))
